In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import os
import argparse
from tqdm.notebook import tqdm
import numpy as np
from itertools import cycle

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
from torchvision.utils import make_grid
import torch.nn.functional as F

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import DataLoader
from sklearn import manifold

import random
from torchvision.utils import save_image

import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

import numpy as np
from itertools import cycle
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torch.distributions.normal import Normal

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

# **Table 1**

In [4]:
is_training = False
print(torch.cuda.is_available())

True


In [5]:
class Encoder(nn.Module):
		def __init__(self):
				super(Encoder, self).__init__()
				self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, bias=True)
				self.linear_mean_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_z = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_z = nn.Linear(in_features=128, out_features=32, bias=True)				
				self.features = nn.Linear(in_features=4608, out_features=512, bias=True)
				self.fc = nn.Linear(in_features=512, out_features=128)
				self.bn_1 = nn.BatchNorm2d(64)
				self.bn_2 = nn.BatchNorm2d(128)
				self.bn_3 = nn.BatchNorm2d(256)
				self.bn_4 = nn.BatchNorm2d(512)
				self.bn_5 = nn.BatchNorm1d(512)
				self.bn_6 = nn.BatchNorm1d(128)
				self.relu = nn.ReLU()
				self.elu = nn.ELU(0.1)
		
		def reparameterize(self, mu, logvar):
				if is_training:
					std = logvar.mul(0.5).exp_()
					eps = torch.randn_like(std)
					if(torch.cuda.is_available()):
						eps = eps.cuda()
					return eps.mul(std).add_(mu)
				else:
					return mu

		def forward(self, x):
			x = self.relu(self.bn_1(self.conv1(x)))
			x = self.relu(self.bn_2(self.conv2(x)))
			x = self.relu(self.bn_3(self.conv3(x)))
			x = self.relu(self.bn_4(self.conv4(x)))
			x = x.view(x.size(0), x.size(1)*x.size(2)*(x.size(3)))
			
			latent_feats = self.relu(self.bn_6(self.fc(self.relu(self.bn_5(self.features(x))))))
			mu_v = self.linear_mean_v(latent_feats)
			logvar_v = self.linear_std_v(latent_feats)
			mu_a = self.linear_mean_a(latent_feats)
			logvar_a = self.linear_std_a(latent_feats)
			mu_z = self.linear_mean_z(latent_feats)
			logvar_z = self.linear_std_z(latent_feats)	 	 
			z_v = self.reparameterize(mu_v, logvar_v)
			z_a = self.reparameterize(mu_a, logvar_a)
			z_z = self.reparameterize(mu_z, logvar_z)	 	 
			return z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z

class Decoder(nn.Module):
		def __init__(self):
				super(Decoder, self).__init__()
				self.inv_fc = nn.Linear(in_features=128, out_features=512, bias=True)
				self.features = nn.Linear(in_features=512, out_features=4608, bias=True)
				self.reconstruct = nn.Linear(in_features=96, out_features=128, bias=True)
				self.deconv_1 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.deconv_2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.deconv_3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.deconv_4 = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, bias=True)
				self.leaky_relu = nn.LeakyReLU(0.2)

				self.bn_1 = nn.BatchNorm1d(num_features=512)
				self.bn_2 = nn.BatchNorm1d(num_features=4608)
				self.bn_3 = nn.BatchNorm2d(num_features=256)
				self.bn_4 = nn.BatchNorm2d(num_features=128)
				self.bn_5 = nn.BatchNorm2d(num_features=64)
				self.bn_6 = nn.BatchNorm2d(num_features=3)

				self.tanh = nn.Tanh()
				self.relu =nn.ReLU()

		def forward(self, z_v, z_a, z_z):
			concat = torch.cat([z_v, z_a], dim=1)
			concat = torch.cat([concat, z_z], dim=1)
			latent = self.relu(self.bn_1(self.inv_fc(self.relu(self.reconstruct(concat)))))
			latent = self.relu(self.bn_2(self.features(latent)))
			x = latent.view(latent.size(0), 512, 3, 3)
			x = self.relu(self.bn_3(self.deconv_1(x)))
			x = self.relu(self.bn_4(self.deconv_2(x)))
			x = self.relu(self.bn_5(self.deconv_3(x)))
			x = self.relu(self.bn_6(self.deconv_4(x)))
			return x

enc = Encoder()
enc.cuda()
z_v,_,_, z_a, _, _, z_z, _, _ = enc.forward(torch.randn(10, 3, 64, 64).cuda())
print('Latents:', z_v.shape, z_a.shape, z_z.shape)

dec = Decoder()
dec.cuda()
i = dec.forward(z_v, z_a, z_z)
print('Recons:', i.shape)

Latents: torch.Size([10, 32]) torch.Size([10, 32]) torch.Size([10, 32])
Recons: torch.Size([10, 3, 64, 64])


In [6]:
def mse_loss(input1, target):
	return torch.sum((input1 - target).pow(2)) / input1.data.nelement()
 
def mae_loss(input1, target):
	return abs(torch.sum((input1 - target).pow(1)) / input1.data.nelement())

def weights_init(layer):
	if isinstance(layer, nn.Conv2d):
		layer.weight.data.normal_(0.0, 0.05)
		layer.bias.data.zero_()
	elif isinstance(layer, nn.BatchNorm2d):
		layer.weight.data.normal_(1.0, 0.02)
		layer.bias.data.zero_()
	elif isinstance(layer, nn.Linear):
		layer.weight.data.normal_(0.0, 0.05)
		layer.bias.data.zero_()


def imshow_grid(images, shape=[2, 8], name='default', save=False):
	"""Plot images in a grid of a given shape."""
	fig = plt.figure(1)
	grid = ImageGrid(fig, 111, nrows_ncols=shape, axes_pad=0.05)

	size = shape[0] * shape[1]
	for i in range(size):
		grid[i].axis('off')
		grid[i].imshow(images[i])  # The AxesGrid object work as a list of axes.

	if save:
		plt.savefig('reconstructed_images/' + str(name) + '.png')
		plt.clf()
	else:
		plt.show()

In [9]:
import json

json_file = '/content/drive/My Drive/LeVAsa_CODS_COMAD/jsons/IMFDB_va.json'

with open(json_file) as f:
  va_dictionary = json.load(f)

import json

json_file = '/content/drive/My Drive/LeVAsa_CODS_COMAD/jsons/IMFDB_labels.json'

with open(json_file) as f:
  label_dictionary = json.load(f)

In [10]:
emotion_mapping = ['ANGER', 'DISGUST', 'FEAR', 'HAPPINESS', 'NEUTRAL', 'SADNESS', 'SURPRISE']

class ImageFolderWithPathsIMFDB(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPathsIMFDB, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        # folder = path[-13:-10]
        fil = path[path.rfind('/')+1:]
        emotion = (emotion_mapping.index(label_dictionary[fil]),)
        tuple_with_path = (original_tuple+(va_dictionary[fil],)+emotion)
        return tuple_with_path

## **Vanilla**

In [14]:
encoder = Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/LeVAsa_CODS_COMAD/experiments/Table_1_continuous/Vanilla_VAE/encoder'))

decoder = Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/LeVAsa_CODS_COMAD/experiments/Table_1_continuous/Vanilla_VAE/decoder'))

batch_size = 2

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/LeVAsa_CODS_COMAD/final_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsIMFDB(data_dir, transform=transform)
print(dset.imgs[:10])
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  X1 = X1.cuda()

s_metric = 0
s_m1 = 0
s_m2 = 0
s_m3 = 0
s_m4 = 0
s_m5 = 0
s_m6 = 0

a_metric = 0
a_m1 = 0
a_m2 = 0
a_m3 = 0
a_m4 = 0
a_m5 = 0
a_m6 = 0

s_v_m = 0
s_a_m = 0
a_v_m = 0
a_a_m = 0

all_z_v = []
all_z_a = []

all_v_labels = []
all_a_labels = []
all_emotion_labels = []

with torch.no_grad():
  for iteration, (data_, _, labels, emotions) in tqdm(enumerate(data_object)):
      image_batch_1 = data_
      X1.copy_(image_batch_1)
      z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z = encoder(X1)

      all_z_v = all_z_v + list(z_v.detach().cpu().numpy())
      all_z_a = all_z_a + list(z_a.detach().cpu().numpy())

      all_v_labels = all_v_labels + list(labels[0].detach().cpu().numpy())
      all_a_labels = all_a_labels + list(labels[1].detach().cpu().numpy())
      all_emotion_labels = all_emotion_labels + list(emotions.detach().cpu().numpy())

      sup_loss = mse_loss(z_v, labels[0].repeat(32, 1).t().cuda()) + mse_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      
      v_l1 = mse_loss(z_v, labels[0].repeat(32, 1).t().cuda())
      # v_l2 = mse_loss(z_a, labels[0].repeat(32, 1).t().cuda())
      # v_l3 = mse_loss(z_z, labels[0].repeat(32, 1).t().cuda())

      # a_l1 = mse_loss(z_v, labels[1].repeat(32, 1).t().cuda())
      a_l2 = mse_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      # a_l3 = mse_loss(z_z, labels[1].repeat(32, 1).t().cuda())


      sup_loss_a = mae_loss(z_v, labels[0].repeat(32, 1).t().cuda()) + mae_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      
      v_l_a1 = mae_loss(z_v, labels[0].repeat(32, 1).t().cuda())
      # v_l_a2 = mae_loss(z_a, labels[0].repeat(32, 1).t().cuda())
      # v_l_a3 = mae_loss(z_z, labels[0].repeat(32, 1).t().cuda())

      # a_l_a1 = mae_loss(z_v, labels[1].repeat(32, 1).t().cuda())
      a_l_a2 = mae_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      # a_l_a3 = mae_loss(z_z, labels[1].repeat(32, 1).t().cuda())

      # s_m1 += v_l1.item()
      # s_m2 += v_l2.item()
      # s_m3 += v_l3.item()
      # s_m4 += a_l1.item()
      # s_m5 += a_l2.item()
      # s_m6 += a_l3.item()

      # a_m1 += v_l_a1.item()
      # a_m2 += v_l_a2.item()
      # a_m3 += v_l_a3.item()
      # a_m4 += a_l_a1.item()
      # a_m5 += a_l_a2.item()
      # a_m6 += a_l_a3.item()

      s_metric += sup_loss.item()
      s_v_m += v_l1.item()
      s_a_m += a_l2.item()

      a_metric += sup_loss_a.item()
      a_v_m += v_l_a1.item()
      a_a_m += a_l_a2.item()

      if(iteration==1000):
          break

print("Combined MSE:", s_metric/(iteration*batch_size), "V MSE:", s_v_m/(iteration*batch_size), "A MSE:", s_a_m/(iteration*batch_size))
print("Combined MAE:", a_metric/(iteration*batch_size), "V MAE:", a_v_m/(iteration*batch_size), "A MAE:", a_a_m/(iteration*batch_size))
# print(s_m1, s_m2, s_m3, s_m4, s_m5, s_m6)
# print(a_m1, a_m2, a_m3, a_m4, a_m5, a_m6)

Loading Dataset...
[('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_1.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_10.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_11.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_12.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_13.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_14.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_15.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_16.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_17.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_ima

Combined MSE: 3.318751613207605 V MSE: 1.8308975290030542 A MSE: 1.4878540842045558
Combined MAE: 0.5575481906207008 V MAE: 0.29410146798955794 A MAE: 0.2634467226311428


## **LeVAsa**

In [16]:
class IMFDB_Reg_Encoder(nn.Module):
		def __init__(self):
				super(IMFDB_Reg_Encoder, self).__init__()
				self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, bias=True)
				self.linear_mean_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_z = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_z = nn.Linear(in_features=128, out_features=32, bias=True)	

				self.features = nn.Linear(in_features=4608, out_features=512, bias=True)
				self.fc = nn.Linear(in_features=512, out_features=128)
				self.bn_1 = nn.BatchNorm2d(64)
				self.bn_2 = nn.BatchNorm2d(128)
				self.bn_3 = nn.BatchNorm2d(256)
				self.bn_4 = nn.BatchNorm2d(512)
				self.bn_5 = nn.BatchNorm1d(512)
				self.bn_6 = nn.BatchNorm1d(128)
				self.relu = nn.ReLU()
				self.elu = nn.ELU(0.1)
		
		def reparameterize(self, mu, logvar):
				if is_training:
					std = logvar.mul(0.5).exp_()
					eps = torch.randn_like(std)
					if(torch.cuda.is_available()):
						eps = eps.cuda()
					return eps.mul(std).add_(mu)
				else:
					return mu

		def forward(self, x):
			x = self.relu(self.bn_1(self.conv1(x)))
			x = self.relu(self.bn_2(self.conv2(x)))
			x = self.relu(self.bn_3(self.conv3(x)))
			x = self.relu(self.bn_4(self.conv4(x)))
			x = x.view(x.size(0), x.size(1)*x.size(2)*(x.size(3)))
			
			latent_feats = self.relu(self.bn_6(self.fc(self.relu(self.bn_5(self.features(x))))))
			mu_v = self.linear_mean_v(latent_feats)
			logvar_v = self.linear_std_v(latent_feats)
			mu_a = self.linear_mean_a(latent_feats)
			logvar_a = self.linear_std_a(latent_feats)
			mu_z = self.linear_mean_z(latent_feats)
			logvar_z = self.linear_std_z(latent_feats)	 	 
			z_v = self.reparameterize(mu_v, logvar_v)
			z_a = self.reparameterize(mu_a, logvar_a)
			z_z = self.reparameterize(mu_z, logvar_z)	 

			return z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z

class IMFDB_Reg_Decoder(nn.Module):
		def __init__(self):
				super(IMFDB_Reg_Decoder, self).__init__()
				self.inv_fc = nn.Linear(in_features=128, out_features=512, bias=True)
				self.features = nn.Linear(in_features=512, out_features=4608, bias=True)
				self.reconstruct = nn.Linear(in_features=96, out_features=128, bias=True)
				self.deconv_1 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.deconv_2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.deconv_3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.deconv_4 = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, bias=True)
				self.leaky_relu = nn.LeakyReLU(0.2)

				self.bn_1 = nn.BatchNorm1d(num_features=512)
				self.bn_2 = nn.BatchNorm1d(num_features=4608)
				self.bn_3 = nn.BatchNorm2d(num_features=256)
				self.bn_4 = nn.BatchNorm2d(num_features=128)
				self.bn_5 = nn.BatchNorm2d(num_features=64)
				self.bn_6 = nn.BatchNorm2d(num_features=3)

				self.tanh = nn.Tanh()
				self.relu =nn.ReLU()

		def forward(self, z_v, z_a, z_z):
			concat = torch.cat([z_v, z_a], dim=1)
			concat = torch.cat([concat, z_z], dim=1)
			latent = self.relu(self.bn_1(self.inv_fc(self.relu(self.reconstruct(concat)))))
			latent = self.relu(self.bn_2(self.features(latent)))
			x = latent.view(latent.size(0), 512, 3, 3)
			x = self.relu(self.bn_3(self.deconv_1(x)))
			x = self.relu(self.bn_4(self.deconv_2(x)))
			x = self.relu(self.bn_5(self.deconv_3(x)))
			x = self.relu(self.bn_6(self.deconv_4(x)))
			return x

enc = IMFDB_Reg_Encoder()
enc.cuda()
z_v,_,_, z_a, _, _, z_z, _, _ = enc.forward(torch.randn(10, 3, 64, 64).cuda())
print('Latents:', z_v.shape, z_a.shape, z_z.shape)

dec = IMFDB_Reg_Decoder()
dec.cuda()
i = dec.forward(z_v, z_a, z_z)
print('Recons:', i.shape)

Latents: torch.Size([10, 32]) torch.Size([10, 32]) torch.Size([10, 32])
Recons: torch.Size([10, 3, 64, 64])


In [18]:
encoder = IMFDB_Reg_Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/LeVAsa_CODS_COMAD/experiments/Table_1_continuous/LeVAsa/reg_encoder_continuous'))

decoder = IMFDB_Reg_Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/LeVAsa_CODS_COMAD/experiments/Table_1_continuous/LeVAsa/reg_decoder_continuous'))

batch_size = 2

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/LeVAsa_CODS_COMAD/final_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsIMFDB(data_dir, transform=transform)
print(dset.imgs[:10])
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  X1 = X1.cuda()

s_metric = 0
s_v_m = 0
s_a_m = 0

a_metric = 0
a_v_m = 0
a_a_m = 0

all_z_v = []
all_z_a = []
all_z_z = []

all_v_labels = []
all_a_labels = []
all_emotion_labels = []

with torch.no_grad():
  for iteration, (data_, _, labels, emotions) in tqdm(enumerate(data_object)):
      image_batch_1 = data_
      X1.copy_(image_batch_1)
      z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z = encoder(X1)

      all_z_v = all_z_v + list(z_v.detach().cpu().numpy())
      all_z_a = all_z_a + list(z_a.detach().cpu().numpy())

      all_v_labels = all_v_labels + list(labels[0].detach().cpu().numpy())
      all_a_labels = all_a_labels + list(labels[1].detach().cpu().numpy())
      all_emotion_labels = all_emotion_labels + list(emotions.detach().cpu().numpy())

      sup_loss = mse_loss(z_v, labels[0].repeat(32, 1).t().cuda()) + mse_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      v_l = mse_loss(z_v, labels[0].repeat(32, 1).t().cuda())
      a_l = mse_loss(z_a, labels[1].repeat(32, 1).t().cuda())

      sup_loss_a = mae_loss(z_v, labels[0].repeat(32, 1).t().cuda()) + mae_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      v_l_a = mae_loss(z_v, labels[0].repeat(32, 1).t().cuda())
      a_l_a = mae_loss(z_a, labels[1].repeat(32, 1).t().cuda())

      s_metric += sup_loss.item()
      s_v_m += v_l.item()
      s_a_m += a_l.item()

      a_metric += sup_loss_a.item()
      a_v_m += v_l_a.item()
      a_a_m += a_l_a.item()

      if(iteration==1000):
          break

print("Combined MSE:", s_metric/(iteration*batch_size), "V MSE:", s_v_m/(iteration*batch_size), "A MSE:", s_a_m/(iteration*batch_size))
print("Combined MAE:", a_metric/(iteration*batch_size), "V MAE:", a_v_m/(iteration*batch_size), "A MAE:", a_a_m/(iteration*batch_size))

Loading Dataset...
[('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_1.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_10.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_11.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_12.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_13.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_14.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_15.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_16.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_17.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_ima

Combined MSE: 0.2080277800738765 V MSE: 0.14363004196394438 A MSE: 0.06439773810993193
Combined MAE: 0.23486813929573 V MAE: 0.1469865681820923 A MAE: 0.08788157111363766


# **Table 2**

In [ ]:
import pickle

data_dir = '/content/drive/My Drive/LeVAsa_CODS_COMAD/experiments/Table_2/'

reg_z_v = pickle.load(open(data_dir+'reg_vae_z_v.pickle', 'rb'))
reg_z_a = pickle.load(open(data_dir+'reg_vae_z_a.pickle', 'rb'))
# reg_z_z = pickle.load(open(data_dir+'reg_vae_z_z.pickle', 'rb'))
reg_v_labels = pickle.load(open(data_dir+'reg_vae_v_labels.pickle', 'rb'))
reg_a_labels = pickle.load(open(data_dir+'reg_vae_a_labels.pickle', 'rb'))
reg_emotion_labels = pickle.load(open(data_dir+'reg_vae_emotion_labels.pickle', 'rb'))

vanilla_z_v = pickle.load(open(data_dir+'vanilla_vae_z_v.pickle', 'rb'))
vanilla_z_a = pickle.load(open(data_dir+'vanilla_vae_z_a.pickle', 'rb'))
# vanilla_z_z = pickle.load(open(data_dir+'vanilla_vae_z_z.pickle', 'rb'))
vanilla_v_labels = pickle.load(open(data_dir+'vanilla_vae_v_labels.pickle', 'rb'))
vanilla_a_labels = pickle.load(open(data_dir+'vanilla_vae_a_labels.pickle', 'rb'))
vanilla_emotion_labels = pickle.load(open(data_dir+'vanilla_vae_emotion_labels.pickle', 'rb'))

In [ ]:
def gridsearch(X_train, y_train):
        tuned_parameters = {
            'hidden_layer_sizes': [(10,), (50,), (90,), (100,)],
            'activation': ['relu'],
            'solver': ['sgd', 'adam'],
        }
        clf = GridSearchCV(
            MLPRegressor(random_state=1), tuned_parameters, scoring="neg_mean_squared_error"
        )
        clf.fit(X_train, y_train)

        y_true, y_pred = y_test, clf.predict(X_test)
        print(mean_squared_error(y_true, y_pred), mean_absolute_error(y_true, y_pred), explained_variance_score(y_test, clf.predict(X_test)), r2_score(y_test, clf.predict(X_test)))#, clf.best_params_, clf.cv_results_)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn import preprocessing

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(reg_z_v)
preprocessing.scale(reg_z_a)

################# Vanilla Valence #########################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_v, vanilla_v_labels, test_size=0.3, random_state=42)

gridsearch(vanilla_z_v, vanilla_v_labels)

# ################# Vanilla Arousal #########################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_a, vanilla_a_labels, test_size=0.3, random_state=42)

gridsearch(vanilla_z_a, vanilla_a_labels)

# ################# Reg Valence #########################

X_train, X_test, y_train, y_test = train_test_split(reg_z_v, reg_v_labels, test_size=0.3, random_state=42)

gridsearch(reg_z_v, reg_v_labels)

# ################# Reg Arousal #########################

X_train, X_test, y_train, y_test = train_test_split(reg_z_a, reg_a_labels, test_size=0.3, random_state=42)

gridsearch(reg_z_a, reg_a_labels)

0.25466069450212503 0.4222959327882577 -0.0034137875637900716 -0.0035008957884801184
0.08920973559293859 0.24493112481672333 -0.008358290286107461 -0.011615461278852157


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.24314636527760874 0.40575350959048934 0.04747090316288405 0.04737829952225481
0.07499406204659798 0.2278122769054959 0.09000087045006844 0.07956007134538179


# **Table 3**

## **Continuous**

In [ ]:
import pickle

data_dir = '/content/drive/My Drive/LeVAsa_CODS_COMAD/experiments/Table_3_continuous/'

reg_z_v = pickle.load(open(data_dir+'reg_vae_z_v.pickle', 'rb'))
reg_z_a = pickle.load(open(data_dir+'reg_vae_z_a.pickle', 'rb'))
reg_z_z = pickle.load(open(data_dir+'reg_vae_z_z.pickle', 'rb'))
reg_v_labels = pickle.load(open(data_dir+'reg_vae_v_labels.pickle', 'rb'))
reg_a_labels = pickle.load(open(data_dir+'reg_vae_a_labels.pickle', 'rb'))
reg_emotion_labels = pickle.load(open(data_dir+'reg_vae_emotion_labels.pickle', 'rb'))

vanilla_z_v = pickle.load(open(data_dir+'vanilla_vae_z_v.pickle', 'rb'))
vanilla_z_a = pickle.load(open(data_dir+'vanilla_vae_z_a.pickle', 'rb'))
vanilla_z_z = pickle.load(open(data_dir+'vanilla_vae_z_z.pickle', 'rb'))
vanilla_v_labels = pickle.load(open(data_dir+'vanilla_vae_v_labels.pickle', 'rb'))
vanilla_a_labels = pickle.load(open(data_dir+'vanilla_vae_a_labels.pickle', 'rb'))
vanilla_emotion_labels = pickle.load(open(data_dir+'vanilla_vae_emotion_labels.pickle', 'rb'))

2002 2002 2002 2002 2002
2002 2002 2002 2002 2002


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(vanilla_z_z)
preprocessing.scale(reg_z_v)
preprocessing.scale(reg_z_a)
preprocessing.scale(reg_z_z)

###################### Vanilla VAE results ################################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_v, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_a, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_z, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla Z MLP: '+str(accuracy_score(y_test, preds)))

###################### LeVAsa results ################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_v, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(reg_z_a, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(reg_z_z, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg Z MLP: '+str(accuracy_score(y_test, preds)))

###################### Concatenated chunk results ################################

zconcat = np.concatenate((vanilla_z_v, vanilla_z_a), axis=1)
X_train, X_test, y_train, y_test = train_test_split(zconcat, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Concat 2 Van MLP: '+str(accuracy_score(y_test, preds)))

zconcat = np.concatenate((reg_z_v, reg_z_a), axis=1)
X_train, X_test, y_train, y_test = train_test_split(zconcat, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Concat 2 Reg MLP: '+str(accuracy_score(y_test, preds)))

concat_van = np.concatenate((vanilla_z_v, vanilla_z_a, vanilla_z_z), axis=1)
concat_reg = np.concatenate((reg_z_v, reg_z_a, reg_z_z), axis=1)

X_train, X_test, y_train, y_test = train_test_split(concat_van, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Concat 3 Van MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(concat_reg, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Concat 3 Reg MLP: '+str(accuracy_score(y_test, preds)))

Vanilla V MLP: 0.2911813643926789
Vanilla A MLP: 0.3227953410981697
Vanilla Z MLP: 0.3211314475873544
Reg V MLP: 0.3627287853577371
Reg A MLP: 0.3560732113144759
Reg Z MLP: 0.3610648918469218
Concat 2 Van MLP: 0.3277870216306156
Concat 2 Reg MLP: 0.3793677204658902
Concat 3 Van MLP: 0.2911813643926789
Concat 3 Reg MLP: 0.33444259567387685


## **Discrete**

In [ ]:
import pickle

data_dir = '/content/drive/My Drive/LeVAsa_CODS_COMAD/experiments/Table_3_discrete/'

reg_z_v = pickle.load(open(data_dir+'reg_vae_z_v_discrete.pickle', 'rb'))
reg_z_a = pickle.load(open(data_dir+'reg_vae_z_a_discrete.pickle', 'rb'))
reg_z_z = pickle.load(open(data_dir+'reg_vae_z_z_discrete.pickle', 'rb'))
reg_emotion_labels = pickle.load(open(data_dir+'reg_vae_emotion_labels_discrete.pickle', 'rb'))

vanilla_z_v = pickle.load(open(data_dir+'vanilla_vae_z_v.pickle', 'rb'))
vanilla_z_a = pickle.load(open(data_dir+'vanilla_vae_z_a.pickle', 'rb'))
vanilla_z_z = pickle.load(open(data_dir+'vanilla_vae_z_z.pickle', 'rb'))
vanilla_emotion_labels = pickle.load(open(data_dir+'vanilla_vae_emotion_labels.pickle', 'rb'))

2002 2002 2002 2002
2002 2002 2002 2002


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(vanilla_z_z)
preprocessing.scale(reg_z_v)
preprocessing.scale(reg_z_a)
preprocessing.scale(reg_z_z)

###################### Vanilla VAE results ################################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_v, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_a, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_z, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla Z MLP: '+str(accuracy_score(y_test, preds)))

###################### LeVAsa VAE results ################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_v, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(reg_z_a, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(reg_z_z, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg Z MLP: '+str(accuracy_score(y_test, preds)))

###################### Concatenated chunk results ################################

zconcat = np.concatenate((vanilla_z_v, vanilla_z_a), axis=1)
X_train, X_test, y_train, y_test = train_test_split(zconcat, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Concat 2 Van MLP: '+str(accuracy_score(y_test, preds)))

zconcat = np.concatenate((reg_z_v, reg_z_a), axis=1)
X_train, X_test, y_train, y_test = train_test_split(zconcat, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Concat 2 Reg MLP: '+str(accuracy_score(y_test, preds)))

concat_van = np.concatenate((vanilla_z_v, vanilla_z_a, vanilla_z_z), axis=1)
concat_reg = np.concatenate((reg_z_v, reg_z_a, reg_z_z), axis=1)

X_train, X_test, y_train, y_test = train_test_split(concat_van, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Concat 3 Van MLP: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(concat_reg, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Concat 3 Reg MLP: '+str(accuracy_score(y_test, preds)))

Vanilla V MLP: 0.30615640599001664
Vanilla A MLP: 0.27454242928452577
Vanilla Z MLP: 0.24126455906821964
Reg V MLP: 0.3594009983361065
Reg A MLP: 0.2995008319467554
Reg Z MLP: 0.2928452579034942
Concat 2 Van MLP: 0.24958402662229617


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Concat 2 Reg MLP: 0.33444259567387685
Concat 3 Van MLP: 0.2628951747088186
Concat 3 Reg MLP: 0.2928452579034942
